# 1) 데이터 준비와 전처리

## 1-1. 평점 데이터 불러오기


In [2]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291



## 1-2. 평점 3점 이상만 남기기

In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


## 1-3. 컬럼 이름 변경: rating> count 

In [5]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [6]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

## 1-4. 영화데이터 불러오기

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
# timestamp 따로 저장, ratings에서 삭제
timestamp = ratings['timestamp']
del ratings['timestamp']

# genre 따로 저장, movies에서 삭제
genre = movies['genre']
del movies['genre']

In [15]:
ratings.head(5)


,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


# 2. 데이터 분석


In [17]:
# movie_id를 가지고 영화 제목 알아내기
# 영화 제목으로 movie_id 찾기

idx_to_movie = {}
movie_to_idx = {}

for i in range(movies.shape[0]):
    idx_to_movie[movies['movie_id'][i]] = movies['title'][i]
    movie_to_idx[movies['title'][i]] = movies['movie_id'][i]

print(idx_to_movie[300])
print(movie_to_idx['Quiz Show (1994)'])

Quiz Show (1994)
300


In [18]:
# movie_id == 300 확인. 일치!

movies[movies['movie_id']==300]

,movie_id,title
297,300,Quiz Show (1994)


In [83]:

# title == 'Quiz Show (1994)' 확인. 일치!

movies[movies['title']=='Quiz Show (1994)']

,movie_id,title
297,300,Quiz Show (1994)


# 2-2. 유니크한 영화, 사용자 수


In [20]:
# ratings에 있는 유니크한 영화 개수
print('유니크한 영화 개수는 :', ratings['movie_id'].nunique(), '개')

# rating에 있는 유니크한 사용자 수
print('유니크한 사용자 수는 :', ratings['user_id'].nunique(), '명')

유니크한 영화 개수는 : 3628 개
유니크한 사용자 수는 : 6039 명


# 2-3. 가장 인기 있는 영화 30개(인기순)


In [21]:
# ratings 데이터 프레임에서 하나의 movie_id에 얼마나 많은 user_id가 있는지 합계 구한뒤
# 내림차순으로 정렬해서 30개만 보여주기

grouped = ratings['user_id'].groupby(ratings['movie_id']).sum()
top_30_movies_id = grouped.sort_values(ascending=False)[:30]
for i in range(len(top_30_movies_id)):
    print('가장 인기있는 {}순위 영화는: {}'.format(i+1, idx_to_movie[top_30_movies_id.index[i]]))

가장 인기있는 1순위 영화는: American Beauty (1999)
가장 인기있는 2순위 영화는: Star Wars: Episode IV - A New Hope (1977)
가장 인기있는 3순위 영화는: Star Wars: Episode V - The Empire Strikes Back (1980)
가장 인기있는 4순위 영화는: Star Wars: Episode VI - Return of the Jedi (1983)
가장 인기있는 5순위 영화는: Back to the Future (1985)
가장 인기있는 6순위 영화는: Saving Private Ryan (1998)
가장 인기있는 7순위 영화는: Silence of the Lambs, The (1991)
가장 인기있는 8순위 영화는: Raiders of the Lost Ark (1981)
가장 인기있는 9순위 영화는: Sixth Sense, The (1999)
가장 인기있는 10순위 영화는: Terminator 2: Judgment Day (1991)
가장 인기있는 11순위 영화는: Matrix, The (1999)
가장 인기있는 12순위 영화는: Fargo (1996)
가장 인기있는 13순위 영화는: Princess Bride, The (1987)
가장 인기있는 14순위 영화는: Jurassic Park (1993)
가장 인기있는 15순위 영화는: Godfather, The (1972)
가장 인기있는 16순위 영화는: Schindler's List (1993)
가장 인기있는 17순위 영화는: Men in Black (1997)
가장 인기있는 18순위 영화는: Shakespeare in Love (1998)
가장 인기있는 19순위 영화는: Shawshank Redemption, The (1994)
가장 인기있는 20순위 영화는: L.A. Confidential (1997)
가장 인기있는 21순위 영화는: Braveheart (1995)
가장 인기있는 22순위 영화는: E.T. the Extra-Terre

# 2-4. 평점이 높은 영화 30개


In [23]:
ratings.sort_values(by=['count'], axis=0, ascending=False, inplace=True)
ratings.reset_index(drop=True, inplace=True)
ratings.head(10)

,user_id,movie_id,count
0,1,1193,5
1,2339,1952,5
2,4671,1210,5
3,2340,715,5
4,2340,2987,5
5,4671,380,5
6,1293,524,5
7,1293,527,5
8,2339,3929,5
9,2339,3922,5


In [37]:

# 평점이 높은 영화 30개

for i in range(1000):
    print('가장 평점이 높은 영화 {}순위는 : {}'.format(i+1, movies[movies['movie_id'] == ratings.loc[i,'movie_id']].iloc[0,1]))

가장 평점이 높은 영화 1순위는 : One Flew Over the Cuckoo's Nest (1975)
가장 평점이 높은 영화 2순위는 : Midnight Cowboy (1969)
가장 평점이 높은 영화 3순위는 : Star Wars: Episode VI - Return of the Jedi (1983)
가장 평점이 높은 영화 4순위는 : Horseman on the Roof, The (Hussard sur le toit, Le) (1995)
가장 평점이 높은 영화 5순위는 : Who Framed Roger Rabbit? (1988)
가장 평점이 높은 영화 6순위는 : True Lies (1994)
가장 평점이 높은 영화 7순위는 : Rudy (1993)
가장 평점이 높은 영화 8순위는 : Schindler's List (1993)
가장 평점이 높은 영화 9순위는 : Bank Dick, The (1940)
가장 평점이 높은 영화 10순위는 : Bikini Beach (1964)
가장 평점이 높은 영화 11순위는 : Beach Party (1963)
가장 평점이 높은 영화 12순위는 : Fish Called Wanda, A (1988)
가장 평점이 높은 영화 13순위는 : Best in Show (2000)
가장 평점이 높은 영화 14순위는 : Phantom of the Opera, The (1943)
가장 평점이 높은 영화 15순위는 : Duck Soup (1933)
가장 평점이 높은 영화 16순위는 : Being John Malkovich (1999)
가장 평점이 높은 영화 17순위는 : Dirty Dancing (1987)
가장 평점이 높은 영화 18순위는 : Godfather: Part II, The (1974)
가장 평점이 높은 영화 19순위는 : Sleepless in Seattle (1993)
가장 평점이 높은 영화 20순위는 : Toy Story (1995)
가장 평점이 높은 영화 21순위는 : Lethal Weapon 4 (1998)
가장 평점

가장 평점이 높은 영화 217순위는 : Maltese Falcon, The (1941)
가장 평점이 높은 영화 218순위는 : Maltese Falcon, The (1941)
가장 평점이 높은 영화 219순위는 : Dancer in the Dark (2000)
가장 평점이 높은 영화 220순위는 : Casablanca (1942)
가장 평점이 높은 영화 221순위는 : Charade (1963)
가장 평점이 높은 영화 222순위는 : Back to the Future Part III (1990)
가장 평점이 높은 영화 223순위는 : Back to the Future Part II (1989)
가장 평점이 높은 영화 224순위는 : Some Like It Hot (1959)
가장 평점이 높은 영화 225순위는 : Harold and Maude (1971)
가장 평점이 높은 영화 226순위는 : Broadcast News (1987)
가장 평점이 높은 영화 227순위는 : Crying Game, The (1992)
가장 평점이 높은 영화 228순위는 : One Flew Over the Cuckoo's Nest (1975)
가장 평점이 높은 영화 229순위는 : Sophie's Choice (1982)
가장 평점이 높은 영화 230순위는 : Boogie Nights (1997)
가장 평점이 높은 영화 231순위는 : North by Northwest (1959)
가장 평점이 높은 영화 232순위는 : Sixth Sense, The (1999)
가장 평점이 높은 영화 233순위는 : Room with a View, A (1986)
가장 평점이 높은 영화 234순위는 : Unbearable Lightness of Being, The (1988)
가장 평점이 높은 영화 235순위는 : Stand by Me (1986)
가장 평점이 높은 영화 236순위는 : Princess Bride, The (1987)
가장 평점이 높은 영화 237순위는 : Die Hard: With

가장 평점이 높은 영화 464순위는 : Man on the Moon (1999)
가장 평점이 높은 영화 465순위는 : Galaxy Quest (1999)
가장 평점이 높은 영화 466순위는 : Ghostbusters (1984)
가장 평점이 높은 영화 467순위는 : Matrix, The (1999)
가장 평점이 높은 영화 468순위는 : Lethal Weapon 4 (1998)
가장 평점이 높은 영화 469순위는 : Wedding Singer, The (1998)
가장 평점이 높은 영화 470순위는 : Shakespeare in Love (1998)
가장 평점이 높은 영화 471순위는 : Close Encounters of the Third Kind (1977)
가장 평점이 높은 영화 472순위는 : Die Hard: With a Vengeance (1995)
가장 평점이 높은 영화 473순위는 : Bull Durham (1988)
가장 평점이 높은 영화 474순위는 : Back to the Future (1985)
가장 평점이 높은 영화 475순위는 : Smoke Signals (1998)
가장 평점이 높은 영화 476순위는 : Parenthood (1989)
가장 평점이 높은 영화 477순위는 : Blue in the Face (1995)
가장 평점이 높은 영화 478순위는 : There's Something About Mary (1998)
가장 평점이 높은 영화 479순위는 : Star Trek: First Contact (1996)
가장 평점이 높은 영화 480순위는 : Breaking Away (1979)
가장 평점이 높은 영화 481순위는 : Wedding Singer, The (1998)
가장 평점이 높은 영화 482순위는 : War of the Worlds, The (1953)
가장 평점이 높은 영화 483순위는 : There's Something About Mary (1998)
가장 평점이 높은 영화 484순위는 : JFK (1991)
가장

가장 평점이 높은 영화 709순위는 : Little Shop of Horrors, The (1960)
가장 평점이 높은 영화 710순위는 : Falling Down (1993)
가장 평점이 높은 영화 711순위는 : Platoon (1986)
가장 평점이 높은 영화 712순위는 : Lady and the Tramp (1955)
가장 평점이 높은 영화 713순위는 : Diner (1982)
가장 평점이 높은 영화 714순위는 : Breaking the Waves (1996)
가장 평점이 높은 영화 715순위는 : Toy Story 2 (1999)
가장 평점이 높은 영화 716순위는 : Election (1999)
가장 평점이 높은 영화 717순위는 : Monty Python's Life of Brian (1979)
가장 평점이 높은 영화 718순위는 : Three Colors: Blue (1993)
가장 평점이 높은 영화 719순위는 : Saving Private Ryan (1998)
가장 평점이 높은 영화 720순위는 : Shakespeare in Love (1998)
가장 평점이 높은 영화 721순위는 : Little Mermaid, The (1989)
가장 평점이 높은 영화 722순위는 : Fish Called Wanda, A (1988)
가장 평점이 높은 영화 723순위는 : Sleeper (1973)
가장 평점이 높은 영화 724순위는 : Blade Runner (1982)
가장 평점이 높은 영화 725순위는 : Spaceballs (1987)
가장 평점이 높은 영화 726순위는 : Muppet Movie, The (1979)
가장 평점이 높은 영화 727순위는 : Shawshank Redemption, The (1994)
가장 평점이 높은 영화 728순위는 : Me, Myself and Irene (2000)
가장 평점이 높은 영화 729순위는 : Matrix, The (1999)
가장 평점이 높은 영화 730순위는 : Three Colors: Red

가장 평점이 높은 영화 951순위는 : Exorcist, The (1973)
가장 평점이 높은 영화 952순위는 : Prizzi's Honor (1985)
가장 평점이 높은 영화 953순위는 : Star Wars: Episode VI - Return of the Jedi (1983)
가장 평점이 높은 영화 954순위는 : Poseidon Adventure, The (1972)
가장 평점이 높은 영화 955순위는 : Terminator, The (1984)
가장 평점이 높은 영화 956순위는 : Doors, The (1991)
가장 평점이 높은 영화 957순위는 : Grumpy Old Men (1993)
가장 평점이 높은 영화 958순위는 : Christmas Story, A (1983)
가장 평점이 높은 영화 959순위는 : Shining, The (1980)
가장 평점이 높은 영화 960순위는 : Harold and Maude (1971)
가장 평점이 높은 영화 961순위는 : Close Encounters of the Third Kind (1977)
가장 평점이 높은 영화 962순위는 : Sting, The (1973)
가장 평점이 높은 영화 963순위는 : Saboteur (1942)
가장 평점이 높은 영화 964순위는 : Boat, The (Das Boot) (1981)
가장 평점이 높은 영화 965순위는 : Remember the Titans (2000)
가장 평점이 높은 영화 966순위는 : Monty Python and the Holy Grail (1974)
가장 평점이 높은 영화 967순위는 : Full Metal Jacket (1987)
가장 평점이 높은 영화 968순위는 : Snow White and the Seven Dwarfs (1937)
가장 평점이 높은 영화 969순위는 : Godfather: Part II, The (1974)
가장 평점이 높은 영화 970순위는 : GoodFellas (1990)
가장 평점이 높은 영화 971순위는 

# 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가

## 3-1. 내가 선호하는 영화 찾기


In [49]:
# Godfather, Toy Story 영화 찾기

for i in range(movies.shape[0]):
    if movies.iloc[i,1][:9] == 'Godfather':
        print(movies.iloc[i,1])
    elif movies.iloc[i,1][:9] == 'Toy Story':
        print(movies.iloc[i,1])
    

Toy Story (1995)
Godfather, The (1972)
Godfather: Part II, The (1974)
Godfather: Part III, The (1990)
Toy Story 2 (1999)


In [50]:
# 선호하는 영화 5개 리스트

my_favorite_title = ['Toy Story (1995)', 
               'Godfather, The (1972)', 
               'Godfather: Part II, The (1974)',
              'Godfather: Part III, The (1990)',
              'Toy Story 2 (1999)']

# 선호하는 영화 movie_id 리스트 생성
my_favorite_id = []
for i in range(movies.shape[0]):
    if movies.iloc[i,1] in my_favorite_title:
        my_favorite_id.append(movies.iloc[i,0])

print(my_favorite_id)

[1, 858, 1221, 2023, 3114]


## 3-2. 내 아이디 및 영화 데이터 ratings에 추가


In [53]:
print(ratings['user_id'].max()) #기존 아이디 6040까지 있음 #나는 6041로 사용

6041


In [52]:

# user_id == 6041가 위 영화를 5번 봤다고 가정

my_playlist = pd.DataFrame({'user_id':[6041]*5, 'movie_id':my_favorite_id, 'count' : [5]*5 })

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_playlist)
    print('추가!')
ratings[ratings['user_id'] == 6041]

추가!


,user_id,movie_id,count
0,6041,1,5
1,6041,858,5
2,6041,1221,5
3,6041,2023,5
4,6041,3114,5


In [54]:
ratings.reset_index(drop=False, inplace=True) # 인덱스 번호 초기화
ratings.drop('index', axis=1, inplace=True) # 'index' 컬럼 삭제
ratings.tail(10)

,user_id,movie_id,count
836473,4194,661,3
836474,1727,2581,3
836475,1727,174,3
836476,4194,1885,3
836477,3080,3536,3
836478,6041,1,5
836479,6041,858,5
836480,6041,1221,5
836481,6041,2023,5
836482,6041,3114,5


# 4. CSR matrix 제작


In [55]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()
# csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [56]:

# 수치 확인

print(num_user)
print(num_movie)
print(csr_data.shape)

6040
3628
(6042, 3953)


# 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련
## 5-1. AlternatingLeastSquares 모델 설계


In [57]:

from implicit.als import AlternatingLeastSquares
import os
import numpy as np

In [69]:
# Implicit AlternatingLeastSquares 모델의 선언

"""
아래 파라미터의 의미
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 함
"""

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=45, dtype=np.float32)

In [70]:

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [71]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/45 [00:00<?, ?it/s]

# 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악
### Checkpoint
- seulgi 벡터와 Toy story의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

## 6-1.내가 선호하는 5가지 영화 중 하나 비교

In [72]:
seulgi, Toy_story = 6041, movie_to_idx['Toy Story (1995)']
seulgi_vector, Toy_story_vector = als_model.user_factors[seulgi], als_model.item_factors[Toy_story]

print('슝=3')

슝=3


In [73]:
seulgi_vector

array([-0.32229194,  0.456297  ,  0.07856417,  0.55494267,  1.1498803 ,
        0.55265605, -0.3168552 , -0.30412737, -1.1039859 ,  0.561763  ,
        0.27615267, -0.50974023, -0.21736911, -0.33487454, -0.07542691,
       -0.32390878, -0.5717998 ,  0.28777197, -0.5320214 ,  0.89185053,
        0.38398898,  0.24916843, -0.2954159 ,  0.7822348 , -0.03736305,
       -0.3766495 , -0.54098976,  0.5801292 ,  0.12391517, -0.26504552,
       -0.10298435, -0.06154277,  0.1407794 , -0.09522133,  0.03218957,
       -0.6386113 ,  0.27495503, -0.09923497, -0.19858675, -0.06430595,
       -0.11347904, -0.36856377,  0.20419957,  0.46562868, -0.04816927,
        0.10968492,  0.13869992,  0.21327823, -0.21471113, -0.1754983 ,
        0.40323684, -0.3732827 ,  0.5944325 , -0.15909512,  0.15412097,
       -0.42288467,  0.19195981, -0.5213528 ,  0.17371029, -0.3588605 ,
       -0.27630174, -0.03225643, -0.02135175, -0.13393797,  0.00234987,
       -0.7440742 ,  0.28135476,  0.3767482 , -0.38109812,  0.41

In [74]:
Toy_story_vector

array([ 0.0109892 , -0.02574558, -0.0107096 ,  0.01926005,  0.02018043,
        0.04484463, -0.00719087,  0.00628705, -0.03224258,  0.05956692,
       -0.02859599, -0.02030236, -0.00299793, -0.03751009, -0.00643447,
       -0.03305539, -0.02764864,  0.02903303, -0.01587164,  0.03491224,
        0.01481521,  0.00341048,  0.01231281,  0.03923428, -0.01290318,
       -0.03042078, -0.0178925 ,  0.04597338,  0.03023428, -0.02473043,
        0.03216675, -0.00723761,  0.02299444, -0.01113831,  0.03051011,
       -0.04147012,  0.02024338,  0.01399706,  0.02316925,  0.01422602,
        0.0003059 , -0.04098744,  0.0019596 ,  0.04232209, -0.01250611,
       -0.01658627, -0.04164466,  0.01452846, -0.00986117, -0.00503746,
        0.03635066,  0.02410845,  0.06371278, -0.00199554,  0.02426251,
        0.02268956, -0.01054862, -0.00811732, -0.00271354, -0.02811366,
       -0.02755606,  0.01924813, -0.01625499,  0.03231776,  0.02453024,
       -0.04056231, -0.00771034, -0.01669845, -0.01354663,  0.00

In [76]:
# seulgi, Toy_story 내적하는 코드
np.dot(seulgi_vector, Toy_story_vector) # 0.60575163(iterations=15)
# 0.63758004(iterations=45)

0.63758004

낮은 수치가 나오면 factors를 늘리거나 iterations를 늘리기

## 6-2. 다른 영화와의 내적값 비교
 

In [78]:
# Godfather 내적 = 0.9516973

Godfather_1 = movie_to_idx['Godfather, The (1972)']
Godfather_1_vector = als_model.item_factors[Godfather_1]
np.dot(seulgi_vector, Godfather_1_vector) # 0.9516973

0.9516973

In [79]:
# Sixth Sense 내적 = 0.11504405

Sixth_Sense = movie_to_idx['Sixth Sense, The (1999)']
Sixth_Sense_vector = als_model.item_factors[Sixth_Sense]
np.dot(seulgi_vector, Sixth_Sense_vector) # 0.11504405

0.11504405

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받기
- 비슷한 아티스트 찾기

AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾기

In [80]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie] # 위에서 정의한 idx_to_movie를 이용해 movie_id값을 이용해 제목 추출
    return similar_movie

In [81]:
get_similar_movie('Back to the Future (1985)')

['Back to the Future (1985)',
 "Ferris Bueller's Day Off (1986)",
 'When Harry Met Sally... (1989)',
 'Back to the Future Part II (1989)',
 'Ghostbusters (1984)',
 'Big (1988)',
 'Cocoon (1985)',
 'Bull Durham (1988)',
 'E.T. the Extra-Terrestrial (1982)',
 'Fish Called Wanda, A (1988)']

# 8. 내가 가장 좋아할 만한 영화들을 추천받기
- AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 추천받기. filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다

## 8-1.영화 추천 받기

In [86]:
user = 6041
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(2355, 0.44425273),
 (34, 0.30806857),
 (1265, 0.23434031),
 (1953, 0.23114276),
 (588, 0.21244305),
 (1213, 0.20836785),
 (356, 0.17301594),
 (3555, 0.16244414),
 (2761, 0.16002469),
 (2396, 0.15854654),
 (3157, 0.15188716),
 (3362, 0.15133154),
 (1276, 0.1499806),
 (367, 0.1466811),
 (2700, 0.14587586),
 (2712, 0.14402266),
 (3072, 0.1436719),
 (3751, 0.14107387),
 (3386, 0.13786858),
 (1210, 0.1363691)]

In [87]:
rec_list = [idx_to_movie[i[0]] for i in movie_recommended]
rec_list

["Bug's Life, A (1998)",
 'Babe (1995)',
 'Groundhog Day (1993)',
 'French Connection, The (1971)',
 'Aladdin (1992)',
 'GoodFellas (1990)',
 'Forrest Gump (1994)',
 'U-571 (2000)',
 'Iron Giant, The (1999)',
 'Shakespeare in Love (1998)',
 'Stuart Little (1999)',
 'Dog Day Afternoon (1975)',
 'Cool Hand Luke (1967)',
 'Mask, The (1994)',
 'South Park: Bigger, Longer and Uncut (1999)',
 'Eyes Wide Shut (1999)',
 'Moonstruck (1987)',
 'Chicken Run (2000)',
 'JFK (1991)',
 'Star Wars: Episode VI - Return of the Jedi (1983)']

## 8-2. 추천한 영화가 왜 나왔는지 추천 기여도 파악


In [89]:
JFK = movie_to_idx['JFK (1991)']
explain = als_model.explain(user, csr_data, itemid=JFK)

In [92]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Godfather: Part III, The (1990)', 0.05196345374888757),
 ('Toy Story 2 (1999)', 0.03894986335207627),
 ('Godfather: Part II, The (1974)', 0.021006089083314968),
 ('Godfather, The (1972)', 0.020083436227136243),
 ('Toy Story (1995)', 0.005056269549996109)]

Godfather: Part III가 추천에 가장 많은 영향을 끼쳤다.